In [1]:
#hide
import sys
import numpy as np
import matplotlib
from matplotlib import animation, pyplot as plt
import proplot as plot

sys.path.append('/Users/46h/Research/code/accphys/')
from tools import (
    plotting as myplt,
    animation as myanim,
    utils
)
plt.rcParams['animation.html'] = 'jshtml'
plot.rc['figure.facecolor'] = 'white'
plot.rc['savefig.dpi'] = 'figure'

# Particle-in-cell simulation
> An introduction to electromagnetic PIC solvers

- toc: true 
- badges: false
- comments: true
- categories: [physics, accelerators, simulation]
- image: images/chart-preview.png

In my research I utilize [PyORBIT](link), one of many particle-in-cell (PIC) simulation codes for particle accelerators. These codes track a beam of particles though the time-varying electromagnetic fields provided by an accelerator; we call this *external focusing*. In addition to external focusing, there are also *internal* forces to consider caused by the electromagnetic interactions between the beam particles. These are known as *space charge* forces, which refers to the charge density of the beam in free space. This post introduces the problem of understanding the effects of space charge through computer simulation.

## Theoretical model 

We'll use *bunch* to refer to a group of particles in three-dimensionl (3D) space; usually a beam is composed of multiple bunches. We want to track a single bunch. We will use a local cartesian coordinate system whose origin is at the center of the bunch as shown below:

<img src='images_2021-02-15/coordinate_system.png' width=600>

The $s$ coordinate specifies the position of the bunch in the accelerator. Now for the simplifying assumptions. First, we will assume the beam is in free space, so there are no external fields. Second, we will assume the center of the bunch is moving at a constant velocity $\beta c$, where $c$ is the speed of light. In this way we can eliminate the time variable in favor of the $s$ variable by $s = \beta c t$. We can then write $z = s - \beta ct$. Third, we will assume there is no energy spread in the bunch, meaning all the particles move at this velocity. Third, we will ignore the dynamics in the $z$ direction, assuming the bunch has a uniform charge density in this dimension and is infinitely long. We will just look at the motion in a thin transverse ($x$-$y$) slice of the bunch. Fourth, we neglect any magnetic fields generated by the beam. This being said, the equations of motion can be written as 

$$ \mathbf{x}'' = \frac{q}{mc^2\beta^2\gamma^3} \mathbf{E}, \tag{1}$$

where $\mathbf{x}$ is the coordinate vector, $\mathbf{E}$ is the electric field, $\gamma = \sqrt{1 - \beta^2}$, and the prime means differentiation with respect to $s$. 

Let's first address the factor $\gamma^{-3}$ in the equation of motion. This means the space charge force goes to zero as the velocity approaches the speed of light. A simple example which illustrates this is to consider two parallel charges moving at speed $\beta_c$ in the lab frame. Of course there is an electrical repulsion between the charges, but we must also remember that a moving charge creates a *magnetic field* which tries to pull the charges together. This reduces the strength of the electric repulsion between the charges, completely cancelling it at the speed of light. One may ask: what about the rest frame? In this frame the magnetic attraction is zero and the particles repel each other. Well, true. But special relativity says that the electrogmagnetic fields change when we switch our reference frame. Using the transformations defined [here](https://en.wikipedia.org/wiki/Classical_electromagnetism_and_special_relativity), you can quickly prove that 

$$\mathbf{E}_{lab} = \frac{\mathbf{E}_{rest}}{\gamma}. \tag{2}$$

As usual for relativity, the math is simple but the interpretation is difficult. It has to do with the [time dilation](https://en.wikipedia.org/wiki/Time_dilation) which occurs when we transform back to the lab frame. This post is not about relativity, so we'll stop the discussion here. It is an important point to mention, however, because it has real-life consequences. It tells us that space charge is important if 1) the beam is very intense, meaning there are many particles in a small area, or 2) the beam is very energetic, meaning it is moving extremely fast. For example, space charge can usually be ignored in electron beams, which move near the speed of light for very modest energies due to their tiny mass, but is significant in high-intensity, low-energy hadron accelerators such as [FRIB](https://frib.msu.edu), [SNS](https://neutrons.ornl.gov/sns), and [ESS](https://europeanspallationsource.se).

Now we should address the difficulty in determining the evolution of this system: the force on a particle in an $n$-particle bunch depends on the positions of the other $n - 1$ particles in the beam. One approach is to introduce a [distribution function](https://en.wikipedia.org/wiki/Distribution_function_(physics)) $f(\mathbf{x}, \mathbf{v}, t)$ which gives the number of particles in an infinitesimal volume of phase space. The Vlasov-Poisson system of equations determines the evolution of $f$ (as long as we ignore collisions between particles and any magnetic fields):

$$\frac{\partial{f}}{\partial{s}} + \mathbf{x}'\cdot \frac{\partial{f}}{\partial{\mathbf{x}}} + \frac{q}{mc^2\beta^2\gamma^3} \mathbf{E} \cdot \frac{\partial{f}}{\partial{\mathbf{x}'}}. \tag{3}$$

The electric field is obtained from the Poisson equation:

$$ \nabla \cdot \mathbf{E} = -\nabla^2 \phi = \frac{\rho}{\varepsilon_0}. \tag{4}$$

Finally, the charge density $\rho$ is determined by integrating the velocity components out of the distribution function:

$$\rho = q \int{f dx'dy'}. \tag{5}$$

Although these equations are easy to write down, they are generally impossible to solve analytically.

## Computational method

We need to turn to a computer for help. We could try to solve the Vlasov equation directly, but this is very difficult. Alternativly, we could track track each individual particle, computing the forces using direct sums, i.e., the force on particle $i$ would be given by $\sum_{j \ne i}^{n} \mathbf{F}_{ji}$. But this is infeasible with current hardware; the number of particles in the bunch may be on the order of $10^{14}$. How can we get around this? 

In comes the particle-in-cell (PIC) method. It is sort of a combination of the above these two approaches. The idea is to track a group of *macroparticles*, each of which represents a large number of real particles, according to Eq. (1). When calculating the field, however, we will assume a continuous charge density and solve the Poisson equation on a discretized grid. We then reverse the process to obtain the electric field at each macroparticle location, and finally update its position. This cycle is shown below.

<img src='images_2021-02-15/simloop.png' width=600>

In the next sections I will discuss each of the 

## Example applications 